# Challenge 2: Sentiment Analysis

In this challenge we will learn sentiment analysis and practice performing sentiment analysis on Twitter tweets. 

## Introduction

Sentiment analysis is to *systematically identify, extract, quantify, and study affective states and subjective information* based on texts ([reference](https://en.wikipedia.org/wiki/Sentiment_analysis)). In simple words, it's to understand whether a person is happy or unhappy in producing the piece of text. Why we (or rather, companies) care about sentiment in texts? It's because by understanding the sentiments in texts, we will be able to know if our customers are happy or unhappy about our products and services. If they are unhappy, the subsequent action is to figure out what have caused the unhappiness and make improvements.

Basic sentiment analysis only understands the *positive* or *negative* (sometimes *neutral* too) polarities of the sentiment. More advanced sentiment analysis will also consider dimensions such as agreement, subjectivity, confidence, irony, and so on. In this challenge we will conduct the basic positive vs negative sentiment analysis based on real Twitter tweets.

NLTK comes with a [sentiment analysis package](https://www.nltk.org/api/nltk.sentiment.html). This package is great for dummies to perform sentiment analysis because it requires only the textual data to make predictions. For example:

```python
>>> from nltk.sentiment.vader import SentimentIntensityAnalyzer
>>> txt = "Ironhack is a Global Tech School ranked num 2 worldwide.   Our mission is to help people transform their careers and join a thriving community of tech professionals that love what they do."
>>> analyzer = SentimentIntensityAnalyzer()
>>> analyzer.polarity_scores(txt)
{'neg': 0.0, 'neu': 0.741, 'pos': 0.259, 'compound': 0.8442}
```

In this challenge, however, you will not use NLTK's sentiment analysis package because in your Machine Learning training in the past 2 weeks you have learned how to make predictions more accurate than that. The [tweets data](https://www.kaggle.com/kazanova/sentiment140) we will be using today are already coded for the positive/negative sentiment. You will be able to use the Naïve Bayes classifier you learned in the lesson to predict the sentiment of tweets based on the labels.

## Conducting Sentiment Analysis

### Loading and Exploring Data

The dataset we'll be using today is located in the lab directory named `Sentiment140.csv.zip`. You need to unzip it into a `.csv` file. Then in the cell below, load and explore the data.

*Notes:* 

* The dataset was downloaded from [Kaggle](https://www.kaggle.com/kazanova/sentiment140). We made a slight change on the original data so that each column has a label.

* The dataset is huuuuge (1.6m tweets). When you develop your data analysis codes, you can sample a subset of the data (e.g. 20k records) so that you will save a lot of time when you test your codes.

In [22]:
import pandas as pd
import re
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords

In [20]:
# Load the dataset
df = pd.read_csv('sample_tweets.csv')
df = df.iloc[:20000]

df.head()

,Unnamed: 0,target,id,date,flag,user,text
0,753159,0,2286791296,Mon Jun 22 16:55:55 PDT 2009,NO_QUERY,iLadySyncere,@oldskool90 whats wrong?
1,658158,0,2241365222,Fri Jun 19 10:51:43 PDT 2009,NO_QUERY,Servante,Dont want to Go to work!
2,440909,0,2066865615,Sun Jun 07 11:18:55 PDT 2009,NO_QUERY,Vonnie617,@queizbella awww I miss you too
3,205082,0,1972784708,Sat May 30 09:52:54 PDT 2009,NO_QUERY,ThomasLierzer,@sueandsteve hahaha not mine
4,1094102,4,1970117918,Sat May 30 02:31:36 PDT 2009,NO_QUERY,RussellBfan90,@TaylaMe3 that's good to hear-i'm warm too


In [15]:
df.shape

(20000, 7)

### Prepare Textual Data for Sentiment Analysis

Now, apply the functions you have written in Challenge 1 to your whole data set. These functions include:

* `clean_up()`

* `tokenize()`

* `stem_and_lemmatize()`

* `remove_stopwords()`

Create a new column called `text_processed` in the dataframe to contain the processed data. At the end, your `text_processed` column should contain lists of word tokens that are cleaned up. Your data should look like below:

![Processed Data](data-cleaning-results.png)

In [23]:
# Define the functions for text preprocessing (you may reuse your existing functions)
def clean_up(s):
    # Remove URLs using regular expressions
    s = re.sub(r'http\S+', '', s)
    
    # Remove special characters and numbers, replace with whitespace
    s = re.sub(r'[^a-zA-Z\s]', ' ', s)
    
    # Convert text to lowercase
    s = s.lower()
    
    return s

def tokenize(s):
    words = word_tokenize(s)
    return words

def stem_and_lemmatize(words):
    stemmer = SnowballStemmer("english")
    lemmatized_words = [stemmer.stem(word) for word in words]
    return lemmatized_words

def remove_stopwords(words):
    stop_words = set(stopwords.words("english"))
    filtered_words = [word for word in words if word not in stop_words]
    return filtered_words

# Apply the text preprocessing functions to the 'text' column of the DataFrame
df['text_processed'] = df['text'].apply(clean_up)
df['text_processed'] = df['text_processed'].apply(tokenize)
df['text_processed'] = df['text_processed'].apply(stem_and_lemmatize)
df['text_processed'] = df['text_processed'].apply(remove_stopwords)

# Display the first few rows of the DataFrame to verify the new 'text_processed' column
print(df[['text', 'text_processed']].head())


                                          text              text_processed
0                    @oldskool90 whats wrong?            [oldskool, wrong]
1                    Dont want to Go to work!       [dont, want, go, work]
2             @queizbella awww I miss you too     [queizbella, awww, miss]
3                @sueandsteve hahaha not mine   [sueandstev, hahaha, mine]
4  @TaylaMe3 that's good to hear-i'm warm too   [taylam, good, hear, warm]


### Creating Bag of Words

The purpose of this step is to create a [bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model) from the processed data. The bag of words contains all the unique words in your whole text body (a.k.a. *corpus*) with the number of occurrence of each word. It will allow you to understand which words are the most important features across the whole corpus.

Also, you can imagine you will have a massive set of words. The less important words (i.e. those of very low number of occurrence) do not contribute much to the sentiment. Therefore, you only need to use the most important words to build your feature set in the next step. In our case, we will use the top 5,000 words with the highest frequency to build the features.

In the cell below, combine all the words in `text_processed` and calculate the frequency distribution of all words. A convenient library to calculate the term frequency distribution is NLTK's `FreqDist` class ([documentation](https://www.nltk.org/api/nltk.html#module-nltk.probability)). Then select the top 5,000 words from the frequency distribution.

In [24]:
from nltk import FreqDist

# Combine all the words in text_processed into a single list
all_words = [word for words in df['text_processed'] for word in words]

# Calculate the frequency distribution of all words
freq_dist = FreqDist(all_words)

# Select the top 5,000 words by frequency
top_words = freq_dist.most_common(5000)

# Extract just the words from the top_words list
top_words = [word[0] for word in top_words]

# Display the top 10 words as an example
print("Top 10 Words by Frequency:")
print(top_words[:10])


Top 10 Words by Frequency:
['go', 'day', 'get', 'good', 'work', 'love', 'like', 'quot', 'got', 'today']


### Building Features

Now let's build the features. Using the top 5,000 words, create a 2-dimensional matrix to record whether each of those words is contained in each document (tweet). Then you also have an output column to indicate whether the sentiment in each tweet is positive. For example, assuming your bag of words has 5 items (`['one', 'two', 'three', 'four', 'five']`) out of 4 documents (`['A', 'B', 'C', 'D']`), your feature set is essentially:

| Doc | one | two | three | four | five | is_positive |
|---|---|---|---|---|---|---|
| A | True | False | False | True | False | True |
| B | False | False | False | True | True | False |
| C | False | True | False | False | False | True |
| D | True | False | False | False | True | False|

However, because the `nltk.NaiveBayesClassifier.train` class we will use in the next step does not work with Pandas dataframe, the structure of your feature set should be converted to the Python list looking like below:

```python
[
	({
		'one': True,
		'two': False,
		'three': False,
		'four': True,
		'five': False
	}, True),
	({
		'one': False,
		'two': False,
		'three': False,
		'four': True,
		'five': True
	}, False),
	({
		'one': False,
		'two': True,
		'three': False,
		'four': False,
		'five': False
	}, True),
	({
		'one': True,
		'two': False,
		'three': False,
		'four': False,
		'five': True
	}, False)
]
```

To help you in this step, watch the [following video](https://www.youtube.com/watch?v=-vVskDsHcVc) to learn how to build the feature set with Python and NLTK. The source code in this video can be found [here](https://pythonprogramming.net/words-as-features-nltk-tutorial/).

[![Building Features](building-features.jpg)](https://www.youtube.com/watch?v=-vVskDsHcVc)

In [25]:
# Define a function to create a feature set for a single tweet
def tweet_features(tweet_words, top_words):
    features = {}
    for word in top_words:
        features[word] = (word in tweet_words)
    return features

# Create a list to store the feature sets in the required format
feature_sets = []

# Iterate through the dataset and create feature sets for each tweet
for i, row in df.iterrows():
    tweet_words = row['text_processed']  # Replace with the correct column name
    sentiment = row['target'] == 4  # Replace with the correct column name
    features = tweet_features(tweet_words, top_words)
    feature_sets.append((features, sentiment))

# Display the first few feature sets as an example
print("Sample Feature Sets:")
for i in range(5):
    print(feature_sets[i])

Sample Feature Sets:
({'go': False, 'day': False, 'get': False, 'good': False, 'work': False, 'love': False, 'like': False, 'quot': False, 'got': False, 'today': False, 'time': False, 'one': False, 'lol': False, 'back': False, 'thank': False, 'want': False, 'miss': False, 'u': False, 'see': False, 'realli': False, 'im': False, 'think': False, 'know': False, 'feel': False, 'amp': False, 'still': False, 'watch': False, 'night': False, 'hope': False, 'need': False, 'make': False, 'new': False, 'well': False, 'look': False, 'na': False, 'come': False, 'much': False, 'oh': False, 'home': False, 'twitter': False, 'morn': False, 'wish': False, 'great': False, 'sad': False, 'tomorrow': False, 'last': False, 'sleep': False, 'bad': False, 'wait': False, 'haha': False, 'onli': False, 'follow': False, 'fun': False, 'whi': False, 'tonight': False, 'week': False, 'happi': False, 'right': False, 'thing': False, 'would': False, 'sorri': False, 'tri': False, 'friend': False, 'better': False, 'way': Fal

### Building and Traininng Naive Bayes Model

In this step you will split your feature set into a training and a test set. Then you will create a Bayes classifier instance using `nltk.NaiveBayesClassifier.train` ([example](https://www.nltk.org/book/ch06.html)) to train with the training dataset.

After training the model, call `classifier.show_most_informative_features()` to inspect the most important features. The output will look like:

```
Most Informative Features
	    snow = True            False : True   =     34.3 : 1.0
	  easter = True            False : True   =     26.2 : 1.0
	 headach = True            False : True   =     20.9 : 1.0
	    argh = True            False : True   =     17.6 : 1.0
	unfortun = True            False : True   =     16.9 : 1.0
	    jona = True             True : False  =     16.2 : 1.0
	     ach = True            False : True   =     14.9 : 1.0
	     sad = True            False : True   =     13.0 : 1.0
	  parent = True            False : True   =     12.9 : 1.0
	  spring = True            False : True   =     12.7 : 1.0
```

The [following video](https://www.youtube.com/watch?v=rISOsUaTrO4) will help you complete this step. The source code in this video can be found [here](https://pythonprogramming.net/naive-bayes-classifier-nltk-tutorial/).

[![Building and Training NB](nb-model-building.jpg)](https://www.youtube.com/watch?v=rISOsUaTrO4)

In [26]:
import random
import nltk

# Shuffle the feature sets to randomize the order
random.shuffle(feature_sets)

# Split the feature sets into a training set (80%) and a test set (20%)
split_index = int(0.8 * len(feature_sets))
train_set = feature_sets[:split_index]
test_set = feature_sets[split_index:]

# Create a Naive Bayes classifier and train it with the training set
classifier = nltk.NaiveBayesClassifier.train(train_set)

# Inspect the most informative features
print("Most Informative Features:")
classifier.show_most_informative_features(10)

Most Informative Features:
Most Informative Features
                 headach = True            False : True   =     35.5 : 1.0
            followfriday = True             True : False  =     27.6 : 1.0
                   upset = True            False : True   =     19.9 : 1.0
                   tummi = True            False : True   =     18.6 : 1.0
                    sigh = True            False : True   =     16.6 : 1.0
                     ach = True            False : True   =     15.3 : 1.0
                     sad = True            False : True   =     14.6 : 1.0
                frustrat = True            False : True   =     13.4 : 1.0
                unfortun = True            False : True   =     13.1 : 1.0
                 stomach = True            False : True   =     12.1 : 1.0


In [34]:
# Example tweet
example_tweet = input("Enter an example tweet: ")

# Preprocess the example tweet
cleaned_example_tweet = clean_up(example_tweet)
tokenized_example_tweet = tokenize(cleaned_example_tweet)
stemmed_and_lemmatized_example_tweet = stem_and_lemmatize(tokenized_example_tweet)

# Create a feature set for the example tweet
example_features = {word: (word in stemmed_and_lemmatized_example_tweet) for word in top_words}

# Predict the sentiment of the example tweet
sentiment_label = classifier.classify(example_features)

# Map the sentiment label to "Positive" or "Negative"
if sentiment_label == True:
    sentiment = "Positive"
else:
    sentiment = "Negative"

# Display the result
print("Example Tweet:", example_tweet)
print("Predicted Sentiment:", sentiment)

Enter an example tweet: i gonna kill myself ahahah !
Example Tweet: i gonna kill myself ahahah !
Predicted Sentiment: Negative


### Testing Naive Bayes Model

Now we'll test our classifier with the test dataset. This is done by calling `nltk.classify.accuracy(classifier, test)`.

As mentioned in one of the tutorial videos, a Naive Bayes model is considered OK if your accuracy score is over 0.6. If your accuracy score is over 0.7, you've done a great job!

In [27]:
accuracy = nltk.classify.accuracy(classifier, test_set)

print("Accuracy:", accuracy)

Accuracy: 0.72625


## Bonus Question 1: Improve Model Performance

If you are still not exhausted so far and want to dig deeper, try to improve your classifier performance. There are many aspects you can dig into, for example:

* Improve stemming and lemmatization. Inspect your bag of words and the most important features. Are there any words you should furuther remove from analysis? You can append these words to further remove to the stop words list.

* Remember we only used the top 5,000 features to build model? Try using different numbers of top features. The bottom line is to use as few features as you can without compromising your model performance. The fewer features you select into your model, the faster your model is trained. Then you can use a larger sample size to improve your model accuracy score.

In [ ]:
# your code here

## Bonus Question 2: Machine Learning Pipeline

In a new Jupyter Notebook, combine all your codes into a function (or a class). Your new function will execute the complete machine learning pipeline job by receiving the dataset location and output the classifier. This will allow you to use your function to predict the sentiment of any tweet in real time. 

In [ ]:
# your code here

## Bonus Question 3: Apache Spark

If you have completed the Apache Spark advanced topic lab, what you can do is to migrate your pipeline from local to a Databricks Notebook. Share your notebook with your instructor and classmates to show off your achievements!

In [ ]:
# your code here